# Keyword_extraction_with_Lasso

small naver news로부터 명사로 이뤄진 term frquency matrix를 만든 뒤, L1 regularization logistic regression을 이용하여 키워드를 추출한다

명사 추출을 위해 soynlp의 LRNounExtractor를 사용

In [1]:
from soynlp.utils import DoublespaceLineCorpus

corpus_path = "/home/paulkim/workspace/python/Korean_NLP/data/small_naver_news/processed/corpus.txt"
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)
len(corpus)

8818

noun score threshold는 0.2 이상, min count는 10 이상인 명사만을 선택하여 custom_tokenizer를 만듬

In [2]:
from soynlp.noun import LRNounExtractor

noun_extrator = LRNounExtractor()
nouns = noun_extrator.train_extract(corpus, min_count=10, minimum_noun_score=0.2)

used default noun predictor; Sejong corpus predictor
used noun_predictor_sejong
2398 r features was loaded
scanning completed
(L,R) has (6862, 3655) tokens
building lr-graph completed

In [3]:
len(nouns)

2046

parse_noun은 주어진 token에 대하여 어절의 왼쪽에 명사가 존재할 경우 이를 잘라내주는 함수임. 만약 '뉴스기자가'라는 어절에 대해 '뉴스'와 '뉴그시자' 두 명사가 모두 존재한다면 길이가 더 긴 '뉴스기자'를 return함. 이를 위하여 reversed(range)를 이용하여 길이의 역순으로 명사를 선택했음

문서가 주어지면 추출된 명사만을 출력하는 custom_tokenize를 만듬. 이를 이용하면 tokenize, part of speech tagging, 이후 명사만 추출하는 과정을 거친 것과 같음

In [4]:
def custom_tokenize(doc):    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            # soynlp.noun.LRNounExtractor객체로 만들어진 사전
            if subword in nouns:
                return subword
        return ''
    
    words = [parse_noun(token) for token in doc.split()]
    words = [word for word in words if word]
    return words

custom_tokenize('국정에 관련된 뉴스입니다')

['국정', '관련', '뉴스']

## Term frequency matrix 만들기
sklearn의 CountVectorizer를 이용하여 term frequency matrix를 만든다

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=custom_tokenize)

corpus.iter_sent = False
x = vectorizer.fit_transform(corpus)
x

<1355x2028 sparse matrix of type '<class 'numpy.int64'>'
	with 44172 stored elements in Compressed Sparse Row format>

In [6]:
index2word = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
index2word = [word for word, i in index2word]
print(index2word[15:20])

['"앞', '"앞으', '"오늘', '"올', '"우리']


In [7]:
print(index2word[50:55])

['130주년', '13일', '14', '14일', '15일']


## Classification을 위한 positive / negative set

국회라는 단어가 들어간 문서와 그렇지 않은 문서로 1355개의 문서의 class를 나눈 뒤, 이를 구분할 수 있는 classifier를 학습하자

In [8]:
aspect_word = '국회'
aspect_id = vectorizer.vocabulary_.get(aspect_word, -1)

print("%s id = %d"%(aspect_word, aspect_id))
print('num of doc (%s) = %d'%(aspect_word, len(x[:, aspect_id].nonzero()[0])))

국회 id = 384
num of doc (국회) = 208


sparse matrix는 nonzero()라는 함수가 있으며, matrix에서 값이 0이 아닌 위치를 (row id list, column id list)로 나타내줌. aspect_id에 해당하는 컬럼을 떼어냈기 때문에 submatrix의 모양이 (1355, 1)이 되었음

In [9]:
print(x[:, aspect_id].shape)
from pprint import pprint
pprint(x[:, aspect_id].nonzero())

(1355, 1)
(array([   2,   16,   18,   20,   27,   33,   39,   52,   62,   76,   85,
         89,  102,  105,  111,  112,  129,  137,  151,  163,  176,  215,
        218,  220,  223,  224,  226,  240,  245,  248,  254,  255,  292,
        298,  299,  304,  342,  343,  356,  357,  359,  365,  372,  384,
        392,  405,  409,  425,  426,  428,  431,  432,  438,  443,  457,
        463,  464,  467,  470,  499,  500,  503,  507,  515,  516,  520,
        521,  526,  543,  548,  555,  557,  564,  566,  568,  571,  576,
        581,  597,  598,  606,  607,  620,  628,  635,  642,  652,  663,
        667,  672,  679,  687,  695,  697,  701,  707,  710,  712,  715,
        716,  733,  735,  736,  738,  744,  750,  751,  754,  755,  759,
        769,  773,  776,  780,  781,  802,  817,  825,  827,  831,  857,
        859,  863,  870,  879,  881,  890,  894,  896,  911,  916,  931,
        934,  948,  955,  958,  959,  969,  990,  998, 1002, 1006, 1008,
       1009, 1011, 1017, 1031, 1033, 103

'국회'란 단어가 들어간 문서들의 리스트를 sparse matrix에서 가져왔음. 이를 이용하여 각 문서에 '국회'라는 단어가 들어있으면 1, 아니면 -1인 label list y를 만든다

In [10]:
pos_idx = set(x[:, aspect_id].nonzero()[0]) # in함수는 list보다 set이 빠름
y = [1 if i in pos_idx else -1 for i in range(x.shape[0])]

print('x shape = %s, len(y) = %d, num_pos = %d'%(str(x.shape), len(y), len(pos_idx)))

x shape = (1355, 2028), len(y) = 1355, num_pos = 208


## L1 (Lasso)를 이용한 keyword extraction
L1 regularization을 이용함. 이는 '국회'가 들어간 문서와 아닌 문서를 구분하기 위한 최소한의 features(=nouns)를 선택하는 효과가 있음

여기서 정의하는 키워드의 기준은, 하난의 class set과 다른 class set을 구분할 수 있는 최소한의 단어집합임. 키워드를 추출하기 위함이기 때문에 이번에는 x_train 대신, '국회'라는 단어가 포함된 term frequency matrix인 x를 사용함

In [11]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1')
logistic_l1.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

L1의 regularization cost에 따른 키워드 추출의 경향을 확인함. cost별로 beta의 크기가 큰 top 50개의 키워드들을 top50 dict에 저장해둠

costs는 1/C의 값임. C = 500이면 lambda = 1/500이므로, regularization이 매우 작게 됨을 의미함

In [12]:
costs = [500, 200, 100, 50]
top50 = {}

for cost in costs:
    logistic_l1 = LogisticRegression(penalty='l1', C=cost)
    logistic_l1.fit(x, y)
    
    coef_l1 = logistic_l1.coef_.reshape(-1)
    beta_l1 = [(index2word[i], coef) for i, coef in enumerate(coef_l1)]
    beta_l1 = sorted(beta_l1, key=lambda x:x[1], reverse=True)
    top50[cost] = beta_l1[:50]
    print('done with cost = %.1f'%cost)

done with cost = 500.0
done with cost = 200.0
done with cost = 100.0
done with cost = 50.0


Regularization이 강하게 되면서, 국회라는 단어가 들어간 문서집합을 설명하기 위해 선택한 단어의 갯수가 줄어듬. 이를 국회라는 문서가 들어간 문서집합의 키워드로 추출할 수 있음

In [13]:
for top in range(50):
    message = '\t'.join(['%10s'%(top50[cost][top][0] if top50[cost][top][1] > 0.001 else '') for cost in costs])
    print(message)

        국회	        국회	        국회	        국회
    최고위원회의	        배재	        배재	     서울구치소
   대표(오른쪽)	    최고위원회의	      당대표실	        의원
        배재	   대표(오른쪽)	    최고위원회의	       최순실
        고발	      진상규명	      국정조사	        의혹
      국조특위	      국조특위	      진상규명	       청문회
      '최순실	        고발	      원내대표	        자택
      의원회관	      당대표실	        의원	        실장
      원내대표	        사표	       최순실	        총장
 국정조사특별위원회	        확보	        의혹	       문체부
      국정농단	        오전	        자택	      원내대표
        사표	        의원	        사표	        대표
       구치소	     서울구치소	       의원들	          
      당대표실	        현장	   대표(오른쪽)	          
      진상규명	       최순실	        현장	          
        이혼	        탈당	     서울구치소	          
        공모	        총장	        합병	          
        귀국	      압수수색	        실장	          
      새누리당	       공무원	        총장	          
        "내	        실장	        오전	          
        탈당	      원내대표	      국정농단	          
        현장	       청문회	        확보	          
       최순실	       대통령	      새누리당